# Summarization of financial data using a Large Language Model (LLM)

This notebook aims to provide an introduction to documenting an LLM using the ValidMind Developer Framework. The use case presented is a summarization of financial news (https://huggingface.co/datasets/cnn_dailymail).

- Initializing the ValidMind Developer Framework
- Running a test various tests to quickly generate documentation about the data and model

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install -q validmind

Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "2494c3838f48efe590d531bfe225d90b",
  api_secret = "4f692f8161f128414fef542cab2a4e74834c75d01b3a8e088a1834f2afcfe838",
  project = "clmotr3oa000aesy6c54wqb6g"
)

2023-10-05 21:29:45,065 - INFO(validmind.api_client): Connected to ValidMind. Project: [12] Credit Risk Scorecard - Initial Validation (clmotr3oa000aesy6c54wqb6g)


## Helper functions

Let's define the following functions to help visualize datasets with long text fields.

In [3]:
import textwrap

from IPython.display import display, HTML
from tabulate import tabulate

def _format_cell_text(text, width=50):  
    """Private function to format a cell's text."""
    return '\n'.join([textwrap.fill(line, width=width) for line in text.split('\n')])

def _format_dataframe_for_tabulate(df):
    """Private function to format the entire DataFrame for tabulation."""
    df_out = df.copy()
    
    # Format all string columns
    for column in df_out.columns:
        if df_out[column].dtype == object:  # Check if column is of type object (likely strings)
            df_out[column] = df_out[column].apply(_format_cell_text)
    return df_out

def _dataframe_to_html_table(df):
    """Private function to convert a DataFrame to an HTML table."""
    headers = df.columns.tolist()
    table_data = df.values.tolist()
    return tabulate(table_data, headers=headers, tablefmt="html")

def display_formatted_dataframe(df, num_rows=None):
    """Primary function to format and display a DataFrame."""
    if num_rows is not None:
        df = df.head(num_rows)
    formatted_df = _format_dataframe_for_tabulate(df)
    html_table = _dataframe_to_html_table(formatted_df)
    display(HTML(html_table))

## Load data

### CNN dataset

The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

In [4]:
from datasets import load_dataset

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')

train_df = cnn_dataset.data['train'].to_pandas()
test_df = cnn_dataset.data['test'].to_pandas()
val_df = cnn_dataset.data['validation'].to_pandas()

train_df = train_df[['article','highlights']]
test_df = test_df[['article','highlights']]

train_df = train_df.head(5)
test_df = test_df.head(5)

In [5]:
display_formatted_dataframe(train_df, num_rows=2)

In [6]:
display_formatted_dataframe(test_df, num_rows=2)

article,highlights
"(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed ""in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014."" Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, said it was a move toward greater justice. ""As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice,"" he said, according to an ICC news release. ""Indeed, today brings us closer to our shared goals of justice and peace."" Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. ""As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly,"" she said. Rights group Human Rights Watch welcomed the development. ""Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court's treaty should speak out to welcome its membership,"" said Balkees Jarrah, international justice counsel for the group. ""What's objectionable is the attempts to undermine international justice, not Palestine's decision to join a treaty to which over 100 countries around the world are members."" In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it ""strongly"" disagreed with the court's decision. ""As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC,"" the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. ""We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace,"" it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as ""Palestine."" While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would ""conduct its analysis in full independence and impartiality."" The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN's Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.","Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June . Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis ."
"(CNN)Never mind cats having nine lives. A stray pooch in Washington State has used up at least three of her

## Model selection

In [7]:
from validmind.models import FoundationModel, Prompt

In [8]:
import os

import dotenv
dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")

In [9]:
import openai

def call_model(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message["content"]

## Prompt engineering

In [10]:
prompt_template = """
You are an AI with expertise in summarizing financial news. 
Your task is to provide a concise summary of the specific news article provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the article.

Article to Summarize:

```
{article}
```

Please respond with a concise summary of the article's main points.
Ensure that your summary is based on the content of the article and not on external information or assumptions.
""".strip()

prompt_variables = ["article"]

## Initialize ValidMind datasets and models

In [11]:
# In this context 'vm_test_ds' is the model evaluation dataset. It is used for predictions
vm_test_ds = vm.init_dataset(
    dataset=test_df,
    text_column="article",
    target_column="highlights",
)

vm_model = FoundationModel(
    predict_fn=call_model,
    prompt=Prompt(
        template=prompt_template,
        variables=prompt_variables,
    ),
    test_ds=vm_test_ds,
)

2023-10-05 21:29:47,970 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-10-05 21:29:47,980 - INFO(validmind.models.foundation): Running predict() for `test_ds`... This may take a while


In [12]:
vm.test_plans.describe_plan("summarization_metrics")

ID,Name,Description,Tests
summarization_metrics,SummarizationMetrics,Test plan for Summarization metrics,RougeMetrics (Metric)TokenDisparity (Metric)BleuScore (Metric)BertScore (Metric)ContextualRecall (Metric)SummarizationPredictions (Metric)


In [13]:
params = {
    "display_limit": 1,
    "text_columns": None
}

metric = vm.tests.run_test("validmind.model_validation.DisplayTextDataset", 
                  dataset=vm_test_ds, 
                  params=params)

In [14]:
params = {
    "display_limit": 1
}

metric = vm.tests.run_test("validmind.model_validation.SummarizationPredictions", 
                  model=vm_model, 
                  params=params)

In [ ]:
#config={
#    "rouge_metric": {
#        "rouge_metrics": ["rouge-1","rouge-2", "rouge-l"],
#    },
#}
#summarization_metrics = vm.run_test_plan("summarization_metrics", 
#                                             model=vm_model,
#                                             config=config)